In [71]:
import json
import os
from geopy.distance import great_circle
from shapely.geometry import LineString
from datetime import datetime


In [72]:
# Ścieżka do folderu z plikami JSON
input_folder_path = r'A:\repo_git\aircraft_signals\dane\json\data'
output_folder_path = r'A:\repo_git\aircraft_signals\dane\json\output'  # Nowa ścieżka do folderu wyjściowego

# Lista nazw plików do przetworzenia
files_to_process = ['AAL639.json','AAL1203.json','AAL1632.json', 'AAL2254.json', 'CBJ480.json', 'DAL446.json', 'DAL1155.json','JBU1197.json','KAL124.json','NKS957.json', 'QFA79.json', 'SWA622.json', 'THY1.json', 'UAL595.json', 'VOZ214.json']

# Utwórz folder wyjściowy, jeśli nie istnieje
os.makedirs(output_folder_path, exist_ok=True)

# Przetwarzanie każdego pliku
for filename in files_to_process:
    file_path = os.path.join(input_folder_path, filename)
    output_filename = f"output_{filename[:-5]}.json"  # Nazwa pliku wynikowego
    
    # Lista do przechowywania punktów
    waypoints = []

    # Wczytaj dane z pliku JSON
    with open(file_path, 'r') as file:
        for line in file:
            try:
                # Usuń niepotrzebne znaki nowej linii lub inne zbędne znaki
                cleaned_line = line.strip()

                # Parsuj linię jako obiekt JSON
                obj = json.loads(cleaned_line)

                # Pobierz potrzebne informacje
                ident = obj['ident']
                reg = obj['reg']
                lat = float(obj['lat'])  # Konwertuj na float
                lon = float(obj['lon'])  # Konwertuj na float
                
                # Sprawdź czy pole alt zawiera wartość
                if 'alt' in obj and obj['alt']:
                    alt = int(obj['alt'])  # Konwertuj na int
                else:
                    alt = -1  # Domyślna wartość, gdy alt jest puste lub nie istnieje
                
                clock = obj['clock']

                # Dodaj punkt do listy waypoints
                waypoints.append({"lat": lat, "lon": lon, "alt": alt, "clock": clock})

            except json.JSONDecodeError as e:
                print(f"Błąd podczas parsowania JSON w pliku {filename}, linia: {cleaned_line[:50]}...")
                print(f"Komunikat błędu: {str(e)}")
                continue

    # Utwórz docelowy obiekt JSON
    output_data = {
        "ident": ident,
        "reg": reg,
        "waypoints": waypoints
    }

    # Konwertuj output_data do formatu JSON
    output_json = json.dumps(output_data, indent=2)

    # Zapisz wynikowy JSON do pliku w folderze wyjściowym
    output_file = os.path.join(output_folder_path, output_filename)
    with open(output_file, 'w') as outfile:
        outfile.write(output_json)

    print(f"Dane zostały zapisane do pliku: {output_file}")


Dane zostały zapisane do pliku: A:\repo_git\aircraft_signals\dane\json\output\output_AAL639.json
Dane zostały zapisane do pliku: A:\repo_git\aircraft_signals\dane\json\output\output_AAL1203.json
Dane zostały zapisane do pliku: A:\repo_git\aircraft_signals\dane\json\output\output_AAL1632.json
Dane zostały zapisane do pliku: A:\repo_git\aircraft_signals\dane\json\output\output_AAL2254.json
Dane zostały zapisane do pliku: A:\repo_git\aircraft_signals\dane\json\output\output_CBJ480.json
Dane zostały zapisane do pliku: A:\repo_git\aircraft_signals\dane\json\output\output_DAL446.json
Dane zostały zapisane do pliku: A:\repo_git\aircraft_signals\dane\json\output\output_DAL1155.json
Dane zostały zapisane do pliku: A:\repo_git\aircraft_signals\dane\json\output\output_JBU1197.json
Dane zostały zapisane do pliku: A:\repo_git\aircraft_signals\dane\json\output\output_KAL124.json
Dane zostały zapisane do pliku: A:\repo_git\aircraft_signals\dane\json\output\output_NKS957.json
Dane zostały zapisane do 

In [73]:
folder_path = r'A:\repo_git\aircraft_signals\dane\json\output'
output_folder_path2 = r'A:\repo_git\aircraft_signals\dane\json'  # Nowa ścieżka do folderu wyjściowego
# Lista nazw plików do przetworzenia
files_to_process = ['output_AAL639.json','output_AAL1203.json','output_AAL1632.json','output_AAL2254.json','output_CBJ480.json','output_DAL446.json','output_DAL1155.json','output_JBU1197.json','output_KAL124.json','output_NKS957.json','output_QFA79.json','output_SWA622.json','output_THY1.json','output_UAL595.json','output_VOZ214.json']

# Lista do przechowywania danych wynikowych
merged_data = []

# Przetwarzanie każdego pliku
for filename in files_to_process:
    file_path = os.path.join(folder_path, filename)
    
    # Wczytaj dane z pliku JSON
    with open(file_path, 'r') as file:
        try:
            # Parsuj dane z pliku JSON
            data = json.load(file)
            
            # Dodaj dane do listy merged_data
            merged_data.append(data)
        
        except json.JSONDecodeError as e:
            print(f"Błąd podczas parsowania JSON w pliku {filename}: {str(e)}")
            continue

# Utwórz ścieżkę dla pliku wynikowego zbiorczego
output_path = os.path.join(output_folder_path2 , 'merged_output.json')

# Zapisz merged_data do pliku JSON
with open(output_path, 'w') as outfile:
    json.dump(merged_data, outfile, indent=2)

print(f"Dane zostały zapisane do pliku: {output_path}")


Dane zostały zapisane do pliku: A:\repo_git\aircraft_signals\dane\json\merged_output.json


In [74]:
import folium
from itertools import cycle

# Ścieżka do pliku JSON
file_path = r'A:\repo_git\aircraft_signals\dane\json\merged_output.json'

# Lista do przechowywania wszystkich obiektów JSON
data = []

# Wczytaj dane z pliku JSON
try:
    with open(file_path, 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    print(f"Plik {file_path} nie został znaleziony.")
    exit(1)
except json.JSONDecodeError as e:
    print(f"Błąd podczas parsowania JSON: {str(e)}")
    exit(1)


In [75]:
import json
from shapely.geometry import LineString

# Ścieżka do pliku JSON
file_path = r'A:\repo_git\aircraft_signals\dane\json\merged_output.json'

# Lista do przechowywania wszystkich obiektów JSON
data = []

# Wczytaj dane z pliku JSON
try:
    with open(file_path, 'r') as file:
        data = json.load(file)
except FileNotFoundError:
    print(f"Plik {file_path} nie został znaleziony.")
    exit(1)
except json.JSONDecodeError as e:
    print(f"Błąd podczas parsowania JSON: {str(e)}")
    exit(1)

# Funkcja do sprawdzania kolizji na podstawie identyfikatorów lotów i punktów przecięcia
def check_collisions(paths, idents):
    collisions = []
    
    for i in range(len(paths)):
        path1 = paths[i]
        ident1 = idents[i]
        reg1 = None
        alt1 = None
        clock1 = None

        for flight_data in data:
            if flight_data['ident'] == ident1:
                reg1 = flight_data.get('reg', 'Nieznana rejestracja')
                alt1 = flight_data['waypoints'][0].get('alt', 'Brak danych')
                clock1 = flight_data['waypoints'][0].get('clock', 'Brak danych')
                break

        for j in range(i + 1, len(paths)):
            path2 = paths[j]
            ident2 = idents[j]
            reg2 = None
            alt2 = None
            clock2 = None

            for flight_data in data:
                if flight_data['ident'] == ident2:
                    reg2 = flight_data.get('reg', 'Nieznana rejestracja')
                    alt2 = flight_data['waypoints'][0].get('alt', 'Brak danych')
                    clock2 = flight_data['waypoints'][0].get('clock', 'Brak danych')
                    break

            # Sprawdź przecięcie ścieżek
            if path1.intersects(path2):
                intersection = path1.intersection(path2)
                if intersection.is_empty:
                    continue
                if intersection.geom_type == 'Point':
                    intersection_lat = intersection.y
                    intersection_lon = intersection.x
                    collisions.append((ident1, reg1, alt1, clock1, ident2, reg2, alt2, clock2, intersection_lat, intersection_lon))
    
    return collisions

# Przygotowanie danych dla funkcji check_collisions
paths = []
idents = []

for flight in data:
    waypoints = flight['waypoints']
    path = LineString([(wp['lon'], wp['lat']) for wp in waypoints])
    paths.append(path)
    idents.append(flight['ident'])

# Wywołanie funkcji sprawdzającej kolizje
collisions = check_collisions(paths, idents)

# Wyświetlenie wyników
if collisions:
    print("Znalezione potencjalne kolizje:")
    for collision in collisions:
        print(f"Lot {collision[0]} ({collision[1]}, alt: {collision[2]}, clock: {collision[3]}) i Lot {collision[4]} ({collision[5]}, alt: {collision[6]}, clock: {collision[7]}) przecinają się w punkcie ({collision[8]}, {collision[9]})")
else:
    print("Nie znaleziono potencjalnych kolizji.")


Znalezione potencjalne kolizje:
Lot SWA622 (N8633A, alt: 1200, clock: 2018-01-01 23:58:20) i Lot AAL2254 (N177AN, alt: 1775, clock: 2018-01-02 00:19:48) przecinają się w punkcie (25.99567699112497, -79.98487707535958)
Lot SWA622 (N8633A, alt: 1200, clock: 2018-01-01 23:58:20) i Lot DAL446 (N355NB, alt: 1300, clock: 2018-01-02 21:32:35) przecinają się w punkcie (31.81216786066269, -94.11626131348235)
Lot AAL1632 (N170US, alt: 3150, clock: 2018-01-01 20:24:14) i Lot DAL446 (N355NB, alt: 1300, clock: 2018-01-02 21:32:35) przecinają się w punkcie (34.999802856572956, -90.02745229745015)
Lot AAL2254 (N177AN, alt: 1775, clock: 2018-01-02 00:19:48) i Lot DAL446 (N355NB, alt: 1300, clock: 2018-01-02 21:32:35) przecinają się w punkcie (40.484711358672875, -73.471327869252)
Lot AAL2254 (N177AN, alt: 1775, clock: 2018-01-02 00:19:48) i Lot JBU1197 (N796JB, alt: 925, clock: 2018-01-01 22:52:08) przecinają się w punkcie (39.609096176139836, -73.34799333743534)
Lot AAL2254 (N177AN, alt: 1775, clock:

In [76]:
map_center = (0, 0)  # Centrum mapy
mymap = folium.Map(location=map_center, zoom_start=2)

# Lista kolorów dla tras samolotów
colors = cycle(['black', 'darkgreen', 'green', 'darkblue', 'darkred', 'gray', 'blue'])

# Utwórz słownik do przechowywania przypisanych kolorów dla każdego identyfikatora
color_map = {}

# Iteruj przez każdy obiekt JSON (reprezentujący trasę jednego samolotu)
for flight_data in data:
    ident = flight_data.get('ident', 'Nieznany ident')
    reg = flight_data.get('reg', 'Nieznana rejestracja')
    waypoints = flight_data.get('waypoints', [])

    # Utwórz listę punktów trasy dla danego samolotu
    flight_route = []
    for waypoint in waypoints:
        lat = waypoint.get('lat')
        lon = waypoint.get('lon')
        alt = waypoint.get('alt')
        clock = waypoint.get('clock')

        # Dodaj punkt trasy do listy
        if lat and lon:
            flight_route.append((lat, lon))

    # Wybierz kolejny kolor dla trasy
    color = next(colors)

    # Dodaj linię łączącą punkty trasy na mapie
    folium.PolyLine(locations=flight_route, color=color, weight=2.5, opacity=1).add_to(mymap)

    # Dodaj marker na początku trasy
    if flight_route:
        start_lat, start_lon = flight_route[0]
        folium.Marker((start_lat, start_lon),
                      popup=f'<b>Start - Ident: {ident}, Reg: {reg}, Clock: {clock}</b>',
                      icon=folium.Icon(color=color)).add_to(mymap)

    # Dodaj marker na końcu trasy
    if flight_route:
        end_lat, end_lon = flight_route[-1]
        folium.Marker((end_lat, end_lon),
                      popup=f'<b>Koniec - Ident: {ident}, Reg: {reg}, Clock: {clock}</b>',
                      icon=folium.Icon(color=color)).add_to(mymap)

    # Przypisz wybrany kolor do identyfikatora
    color_map[ident] = color

# Dodaj markery w miejscach przecięć tras samolotów
for collision in collisions:
    ident1, reg1, alt1, clock1, ident2, reg2, alt2, clock2, intersection_lat, intersection_lon = collision
    popup_content = f"""
    <b>Przecięcie:</b><br>
    Reg1: {reg1}<br>
    Alt1: {alt1}<br>
    Clock1: {clock1}<br>
    <br>
    Reg2: {reg2}<br>
    Alt2: {alt2}<br>
    Clock2: {clock2}<br>
    <br>
    Współrzędne przecięcia:<br>
    Latitude: {intersection_lat}<br>
    Longitude: {intersection_lon}
    """
    folium.Marker((intersection_lat, intersection_lon),
                  popup=popup_content,
                  icon=folium.Icon(color='red')).add_to(mymap)

# Zapisz mapę do pliku HTML
output_map_path = r'A:\repo_git\aircraft_signals\dane\maps\flight_route_map_with_collisions_details.html'
mymap.save(output_map_path)

print(f"Mapa trasy samolotów z oznaczeniem przecięć i rozszerzonymi szczegółami została zapisana do pliku: {output_map_path}")

Mapa trasy samolotów z oznaczeniem przecięć i rozszerzonymi szczegółami została zapisana do pliku: A:\repo_git\aircraft_signals\dane\maps\flight_route_map_with_collisions_details.html
